In [ ]:
#in this workbook, I will take the model generated and apply it against the November data set as the basis of the Kaggle submission

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pickle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
Nov=pd.read_csv("/home/cdsw/Novdata_for_preds.csv")

In [2]:
# Load from file
with open("/home/cdsw/Models/rf_red_1000_min5_v2", 'rb') as file:
    rf2 = pickle.load(file)

In [12]:
#reduce the data down to those which are non null
Nov2=Nov[Nov.PStdMinus1.notnull()]
#extract the target variable
Nov3=Nov2.drop(["Target"],axis="columns")
Nov3=Nov3.drop(Nov3.iloc[:,0:24],axis=1)
#predict the results from the remaining data
yhat2=pd.Series(rf2.predict(Nov3))

In [15]:
round(yhat2,0).astype("int64").describe()

count    61387.000000
mean         0.759933
std          5.216102
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max        453.000000
dtype: float64

In [19]:
#read in the submission format to know what I have to prepare
submission=pd.read_csv("/home/cdsw/01_future-sales/data/test.csv")
submission.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [37]:
#data will be in Nov2
prep=Nov2.loc[:,["shop_id","item_id"]]
prep.reset_index(drop=True,inplace=True)
prep["Sales"]=yhat2

sub2=pd.merge(submission, prep, how="left",on=["shop_id","item_id"])
sub2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 0 to 214199
Data columns (total 5 columns):
ID           214200 non-null int64
shop_id      214200 non-null int64
item_id      214200 non-null int64
identifer    214200 non-null object
Sales        47696 non-null float64
dtypes: float64(1), int64(3), object(1)
memory usage: 9.8+ MB


In [56]:
sub2["item_cnt_month"]=sub2.Sales.fillna(0)
sub3=sub2.drop(sub2.iloc[:,1:5],axis=1)
sub3.to_csv("/home/cdsw/Submission1_Predict48k.csv",index=False)

In [48]:
eg=pd.read_csv("/home/cdsw/01_future-sales/data/sample_submission.csv")
eg.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
